In [11]:
import pandas as pd
import numpy as np
import os 
from datetime import datetime, timedelta
import re
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import DateOffset


기본 메서드

In [12]:
def get_today(form='%Y-%m-%d'):
    mapping = {
        '%Y%m%d': datetime.now().strftime("%Y%m%d"),
        'yyyymmdd': datetime.now().strftime("%Y%m%d"),
        '%Y-%m-%d': datetime.now().strftime("%Y-%m-%d"),
        'yyyy-mm-dd': datetime.now().strftime("%Y-%m-%d"),
        'datetime': datetime.now(),
        '%Y%m%d%H': datetime.now().strftime("%Y%m%d%H"),
    }
    today = mapping[form]
    return today 

def scan_files_including_regex(file_folder, regex, option='name'):
    with os.scandir(file_folder) as files:
        lst = [file.name for file in files if re.findall(regex, file.name)]
    
    mapping = {
        'name': lst,
        'path': [os.path.join(file_folder, file_name) for file_name in lst]
    }
    return mapping[option]

def format_date(date):
    date = date.replace('-', '')
    date = datetime.strptime(date, '%Y%m%d').strftime('%Y-%m-%d')
    return date

def save_df_to_file(df, file_folder, subject, file_memo, file_code,input_date, include_index=False, file_extension='.csv', archive=False, archive_folder='./archive'):
    def get_today(form='%Y%m%d'):
        return datetime.now().strftime(form)
    try:
#         new_folder_path = os.path.join(file_folder, new_folder_name)
#         os.makedirs(new_folder_path, exist_ok=True)
        save_time = get_today()
        file_name = f'dataset-{subject}-{file_memo}-code{file_code}-date{input_date}-save{save_time}{file_extension}'
        file_path = os.path.join(file_folder, file_name)
        if os.path.exists(file_path) and archive:
            df_archive = pd.read_csv(file_path)
            os.makedirs(archive_folder, exist_ok=True)
            archive_file_name = 'archive-' + file_name
            archive_file_path = os.path.join(archive_folder, archive_file_name)
            df_archive.to_csv(archive_file_path, index=False)
            print(f'Archived: {archive_file_path}')
        df.to_csv(file_path, index=include_index, encoding='utf-8-sig')
        print(f'Saved: {file_path}')
    except Exception as e:
        print(f"Error: {e}")

In [60]:
class M8186:
    def __init__(self, fund_code, start_date =None, end_date = None, menu_code = '8186'):
        self.fund_code = fund_code
        self.menu_code = menu_code
        self.start_date = start_date
        self.end_date = end_date
        self.df = None  # 데이터프레임을 위한 초기화

        self.columns_multiindex = ['수정기준가', 'KOSPI지수', 'KOSPI200지수', 'KOSDAQ지수', 'spx']
        self.columns_singleindex = ['수정기준가', 'KOSPI지수']

    def open_df_raw(self):
        lst = scan_files_including_regex(file_folder = './캡스톤데이터2', regex = f'menu{self.menu_code}-code{self.fund_code}')
        lst = sorted(lst, reverse = True)
        file_path = lst[0]
        full_path = os.path.join(os.getcwd(), '캡스톤데이터2', file_path)
        df = pd.read_csv(full_path)

        if df.isnull().all(axis=1).any():  # 데이터프레임에 누락된 값이 있는지 확인
            raise ValueError("데이터 파일에 누락된 부분이 존재합니다. 데이터 확인이 필요합니다.")
        return df
    
    def get_df_ref(self, columns=None):
        self.df = self.open_df_raw()
        default_columns = ['일자', '수정기준가', 'KOSPI지수', 'KOSPI200지수', 'KOSDAQ지수']

        # 전달된 칼럼 리스트가 없으면 기본 칼럼 리스트 사용
        if columns is None:
            columns = default_columns

        # 선택된 칼럼만 데이터프레임에 적용
        self.df = self.df[columns]

        if self.start_date is None:
            self.start_date = self.df['일자'].min()

        if self.end_date is None:
            self.end_date = self.df['일자'].max()

        return self.df

    def open_df_SPX_index_raw(self):
        # 'dataset-index' 폴더에서 'dataset-price-' 패턴을 포함하는 파일 목록을 가져옴
        lst = scan_files_including_regex('./dataset-index', 'dataset-price-')
        lst = sorted(lst, reverse = True)
        file_path = lst[0]
        full_path = os.path.join(os.getcwd(), 'dataset-index', file_path)
        df = pd.read_csv(full_path)

        df['SPX INDEX'] = pd.to_numeric(df['SPX INDEX'], errors='coerce')
        df = df.dropna(subset=['SPX INDEX']).reset_index(drop=True)
        df.rename(columns={'SPX INDEX': 'spx', 'ticker': '일자'}, inplace=True)

        if df.isnull().all(axis=1).any():  # 데이터프레임에 누락된 값이 있는지 확인
            raise ValueError("데이터 파일에 누락된 부분이 존재합니다. 데이터 확인이 필요합니다.")
        return df

    def get_merged_df(self, avoid_nan = True):
        df_ref = self.get_df_ref()

        # open_df_SPX_index_raw에서 반환된 데이터프레임을 가져옴
        df_spx = self.open_df_SPX_index_raw()

        # '일자' 컬럼을 기준으로 두 데이터프레임을 병합
        # how='left' 옵션은 df_ref 데이터프레임을 기준으로 합치기 위함
        self.df = pd.merge(df_ref, df_spx, on='일자', how='left')

        # 비어 있는 값들을 각 열의 바로 앞 행의 값으로 대체
        if avoid_nan:
            self.df.fillna(method='ffill', inplace=True)

        for column in self.df.columns:
            if self.df[column].iloc[0] == 0 or pd.isna(self.df[column].iloc[0]):
                self.df.at[0, column] = self.df[column].iloc[1]
        
        return self.df

    def fill_zero_with_previous(self, columns=None):
        if columns is None:
            columns = self.columns_multiindex
        
        for column in columns:
            self.df[column] = self.df[column].replace(0, None)
            self.df[column] = self.df[column].ffill()
        return self.df

    def convert_to_float(self, columns=None):
        if columns is None:
            columns = self.columns_multiindex

        for column in columns:
            self.df[column] = self.df[column].apply(lambda x: float(x.replace(',', '' )) if isinstance (x,str) else x)
        return self.df 
            
    def filter_by_date_range(self):
        # '일자' 컬럼을 datetime 타입으로 변환
        self.df['일자'] = pd.to_datetime(self.df['일자'])

        # start_date와 end_date를 기준으로 데이터 필터링
        self.df = self.df[(self.df['일자'] >= self.start_date) & (self.df['일자'] <= self.end_date)]
        return self.df

    def calculate_cumulative_return_for_df(self, df, columns = None):
        df = df.copy()  # 명시적으로 데이터프레임 복사본 생성
        if columns is None:
            columns = self.columns_multiindex

        for column_name in columns:
            if column_name in df.columns:
                initial_value = df[column_name].iloc[0]
                updated_values = ((df[column_name] - initial_value) / initial_value) * 100
                updated_values.iloc[0] = 0  # 첫 번째 행의 수익률을 0으로 설정
                df.loc[:, column_name + ' (%)'] = updated_values
        return df

    def get_cumulative_return(self):
        df = self.calculate_cumulative_return_for_df(self.df, columns = self.columns_singleindex)
        cumulative_returns = {}

        for column_name in self.columns_singleindex:
            cumulative_return = df[column_name + ' (%)'].iloc[-1]
            cumulative_returns[column_name] = cumulative_return

        return cumulative_returns
    
    def get_annualized_return(self):
        df = self.calculate_cumulative_return_for_df(self.df, self.columns_singleindex)
        start_date = df['일자'].iloc[0]
        end_date = df['일자'].iloc[-1]
        days = (end_date - start_date).days + 1

        # 연환산 수익률 계산을 위한 딕셔너리 초기화
        annualized_returns = {}

        # days가 365일 이상일 경우에만 연환산 수익률 계산
        if days >= 365:
            for column_name in self.columns_singleindex:
                cumulative_return = df[column_name + ' (%)'].iloc[-1]
                annualized_return = (cumulative_return * 365) / days
                annualized_returns[column_name] = annualized_return
        else:
            # days가 365일 미만일 경우, 연환산 수익률을 계산하지 않음
            for column_name in self.columns_singleindex:
                annualized_returns[column_name] = None

        return annualized_returns

    def get_volatility(self):
        # 변동성 계산을 위한 딕셔너리 초기화
        volatility = {}

        self.daily_returns = self.df[self.columns_singleindex].pct_change()
        self.daily_returns.iloc[0] = 0
        self.daily_returns_std = self.daily_returns.std()

        # 각 칼럼에 대한 변동성 계산
        for column_name in self.columns_singleindex:
            volatility[column_name] = self.daily_returns_std[column_name] * (365 ** 0.5) * 100

        return volatility
    
    def get_sharpe_ratio(self, risk_free_rate=0):
        # 연환산 수익률 계산
        annualized_returns = self.get_annualized_return()

        # 변동성 계산
        volatility = self.get_volatility()

        # Sharpe 비율 계산을 위한 딕셔너리 초기화
        sharpe_ratios = {}

        # 각 칼럼에 대한 Sharpe 비율 계산
        for column_name in self.columns_singleindex:
            if annualized_returns[column_name] is not None and column_name in volatility:
                sharpe_ratios[column_name] = (annualized_returns[column_name] - risk_free_rate) / volatility[column_name]

        return sharpe_ratios    

    # def get_winning_ratio(self):
    #     # columns_singleindex에 대한 일일 차이 계산
    #     daily_diff = self.df[self.columns_singleindex].diff()

    #     # 첫 번째 행을 0으로 설정
    #     daily_diff.iloc[0] = 0

    #     # 0인 행 제거
    #     daily_diff = daily_diff[daily_diff != 0].dropna()

    #     # 승리 비율 계산
    #     winning_ratios = {}
    #     for column in self.columns_singleindex:
    #         # 양수인 값의 개수 계산
    #         positive_count = (daily_diff[column] > 0).sum()
    #         # 전체 유효한 값의 개수 계산
    #         total_count = len(daily_diff[column])
    #         # 승리 비율 계산 및 저장
    #         winning_ratios[column] = (positive_count / total_count) * 100 if total_count > 0 else 0

    #     return winning_ratios

    def get_winning_ratio(self):
        # columns_singleindex에 대한 일일 차이 계산
        daily_diff = self.df[self.columns_singleindex].diff()

        # 첫 번째 행을 0으로 설정
        daily_diff.iloc[0] = 0

        # 승리 비율 계산
        winning_ratios = {}
        for column in self.columns_singleindex:
            # 양수인 값의 개수 계산
            positive_count = (daily_diff[column] > 0).sum()
            # 전체 유효한 값의 개수 계산
            total_count = len(daily_diff[column])
            # 승리 비율 계산 및 저장
            winning_ratios[column] = (positive_count / total_count) * 100 if total_count > 0 else 0

        return winning_ratios


    def get_mdd(self):
        # MDD를 저장할 딕셔너리 초기화
        mdd = {}

        # columns_singleindex 내의 각 칼럼에 대해 MDD 계산
        for column in self.columns_singleindex:
            mdd_values = []
            for index, max_value in self.df[column].items():
                min_value_after = self.df.loc[index:, column].min()
                current_mdd = (max_value - min_value_after) / max_value if max_value > 0 else 0
                mdd_values.append(current_mdd)
  
            # 최대 MDD 값 계산
            max_mdd = max(mdd_values) * 100

            # 각 칼럼별 최대 MDD 저장
            mdd[column] = max_mdd

        return mdd

    def filter_for_period(self, months):
        if months is not None:
            # 현재 가장 최근 날짜를 구함
            df_end_date = self.df['일자'].max()

            # 지정된 개월 수만큼 과거 날짜를 계산
            period_start_date = df_end_date - DateOffset(months=months)

            # period_start_date보다 이전 데이터를 필터링
            filtered_df = self.df[self.df['일자'] >= period_start_date]

            return filtered_df
        else:
            # months가 None이면 전체 데이터프레임 반환
            return self.df


    def generate_period_df(self):
        # self.df의 최대 및 최소 날짜 찾기
        df_start_date = self.df['일자'].min()
        df_end_date = self.df['일자'].max()

        # 가능한 모든 기간을 검사하여 default_periods 설정
        potential_periods = [1, 3, 6, 12, 24, 36, 48, 60]
        default_periods = []

        for period in potential_periods:
            period_start_date = df_end_date - DateOffset(months=period)
            if period_start_date >= df_start_date:
                default_periods.append(period)

        period_dfs = {}  # 각 기간에 해당하는 데이터프레임을 저장할 딕셔너리

        # 각 기간에 대한 데이터프레임 생성
        for period in default_periods:
            period_dfs[f"{period}m"] = self.filter_for_period(period)

        # YTD 데이터프레임 생성
        # 현재 연도 필터링
        current_year = pd.Timestamp.now().year
        current_year_df = self.df[self.df['일자'].dt.year == current_year]

        # 현재 연도 데이터가 1월 1일부터 시작하는지 확인
        if current_year_df['일자'].min() == pd.Timestamp(year=current_year, month=1, day=1):
            period_dfs['YTD'] = current_year_df

        return period_dfs

    def format_period(self, period):
        """
        '기간' 값을 포맷하는 함수. 
        예: '1m' -> '1개월', '12m' -> '1년' 등
        """
        try:
            months = int(period.replace('m', ''))
            if months < 12:
                return f'{months}개월'
            elif months % 12 == 0:
                years = months // 12
                return f'{years}년'
        except ValueError:
            return period  # 만약 다른 형식이라면 원본 값을 반환

    def get_final_cumulative_returns(self, period_dfs):
        final_returns_data = []

        for period, df in period_dfs.items():
            formatted_period = self.format_period(period)  # 기간 포맷 변경
            last_row = df.iloc[-1]
            row_data = {
                '기간': formatted_period,
                '펀드': last_row.get('수정기준가 (%)', None),
                'KOSPI': last_row.get('KOSPI지수 (%)', None),
                'KOSPI200': last_row.get('KOSPI200지수 (%)', None),
                'KOSDAQ': last_row.get('KOSDAQ지수 (%)', None),
                'S&P 500': last_row.get('spx (%)', None)
            }
            final_returns_data.append(row_data)

        # 데이터를 기반으로 새로운 데이터프레임 생성
        final_returns_df = pd.DataFrame(final_returns_data)
        final_returns_df.set_index('기간', inplace=True)

        return final_returns_df
    
    def process_period_dfs(self):
        # 각 기간별 데이터프레임을 생성
        period_dfs = self.generate_period_df()

        # 각 데이터프레임에 대해 누적 수익률 계산
        for period, df in period_dfs.items():
            period_dfs[period] = self.calculate_cumulative_return_for_df(df)

        # 전체 기간에 대한 누적수익률 추가
        period_dfs['설정이후'] = self.calculate_cumulative_return_for_df(self.df)
        
        # 각 기간별 누적수익률의 마지막 값으로 구성된 데이터프레임을 반환
        final_returns_df = self.get_final_cumulative_returns(period_dfs)

        return final_returns_df
    
    def get_investment_performance_df(self):
        # 각 메서드를 호출하여 지표값을 가져옴
        cumulative_returns = self.get_cumulative_return()
        annualized_returns = self.get_annualized_return()
        volatility = self.get_volatility()
        sharpe_ratios = self.get_sharpe_ratio()
        winning_ratios = self.get_winning_ratio()
        mdd = self.get_mdd()

        # 데이터프레임 생성
        summary_df = pd.DataFrame({
            '누적수익률': cumulative_returns,
            '연환산 수익률': annualized_returns,
            '변동성': volatility,
            '샤프비율': sharpe_ratios,
            'Winning Ratio': winning_ratios,
            'MDD': mdd
        })

        # columns_singleindex를 사용하여 행 인덱스 이름 변경
        index_rename_map = {
            self.columns_singleindex[0]: '펀드',
            self.columns_singleindex[1]: 'KOSPI'
        }
        summary_df = summary_df.rename(index=index_rename_map)

        return summary_df

    #기간별 수익률을 위한 메인 메서드 
    def period_cumulative_return(self):
        self.get_merged_df()
        self.filter_by_date_range()
        self.convert_to_float()
        self.fill_zero_with_previous()
        #convert_to_float 이 먼저 실행되고 fill_zero_with_previous 가 실행되어야 함 
        final_returns_df = self.process_period_dfs()

        return final_returns_df

    #성능평가지표를 위한 메인 메서드
    def investment_performance(self):
        self.get_df_ref(['일자']+self.columns_singleindex)
        self.filter_by_date_range()
        self.convert_to_float(self.columns_singleindex)
        self.fill_zero_with_previous(self.columns_singleindex)
        self.calculate_cumulative_return_for_df(self.df, self.columns_singleindex)
        summary_df = self.get_investment_performance_df()

        return summary_df

    # def get_monthly_first_dates(self):
    #     # '일자' 컬럼을 datetime 타입으로 변환
    #     self.df['일자'] = pd.to_datetime(self.df['일자'])

    #     # 월별로 그룹화하여 각 달의 첫날을 찾음
    #     monthly_first_dates = self.df.groupby(self.df['일자'].dt.to_period('M')).agg({'일자': 'first'})

    #     # 월별 첫날만 저장할 딕셔너리 초기화
    #     first_dates_dict = {}

    #     # 각 달의 첫날 추출 및 딕셔너리에 저장
    #     for period, row in monthly_first_dates.iterrows():
    #         year_month_key = f"{period.year}-{str(period.month).zfill(2)}"
    #         first_dates_dict[year_month_key] = row['일자']

    #     return first_dates_dict
    
    # def get_monthly_last_dates(self):
    #     # '일자' 컬럼을 datetime 타입으로 변환
    #     self.df['일자'] = pd.to_datetime(self.df['일자'])

    #     # 월별로 그룹화하여 각 달의 마지막날을 찾음
    #     monthly_last_dates = self.df.groupby(self.df['일자'].dt.to_period('M')).agg({'일자': 'last'})

    #     # 월별 마지막날만 저장할 딕셔너리 초기화
    #     last_dates_dict = {}

    #     # 각 달의 마지막날 추출 및 딕셔너리에 저장
    #     for period, row in monthly_last_dates.iterrows():
    #         year_month_key = f"{period.year}-{str(period.month).zfill(2)}"
    #         last_dates_dict[year_month_key] = row['일자']

    #     return last_dates_dict
    
    # def get_monthly_returns(self):
    #     # 첫날과 마지막날 날짜 가져오기
    #     first_dates = self.get_monthly_first_dates()
    #     last_dates = self.get_monthly_last_dates()

    #     # 월간 수익률 저장할 딕셔너리 초기화
    #     monthly_returns = {}

    #     # 각 달의 첫날과 마지막날 데이터를 이용해 월간 수익률 계산
    #     for key in first_dates.keys():
    #         if key in last_dates:
    #             start_date = first_dates[key]
    #             end_date = last_dates[key]

    #             start_values = self.df.loc[self.df['일자'] == start_date, ['수정기준가', 'KOSPI지수']].values[0]
    #             end_values = self.df.loc[self.df['일자'] == end_date, ['수정기준가', 'KOSPI지수']].values[0]

    #             # 월간 수익률 계산
    #             monthly_return_fund = ((end_values[0] - start_values[0]) / start_values[0]) * 100
    #             monthly_return_kospi = ((end_values[1] - start_values[1]) / start_values[1]) * 100

    #             # 딕셔너리에 저장
    #             monthly_returns[key] = {
    #                 '수정기준가_월간수익률': monthly_return_fund,
    #                 'KOSPI지수_월간수익률': monthly_return_kospi
    #             }

    #     return monthly_returns


    def get_monthly_dates(self):
        # '일자' 컬럼을 datetime 타입으로 변환
        self.df['일자'] = pd.to_datetime(self.df['일자'])

        # 각 달의 마지막날을 찾음
        monthly_last_dates = self.df.groupby(self.df['일자'].dt.to_period('M')).agg({'일자': 'last'}).reset_index(drop=True)

        # 데이터의 첫 값 추가
        first_date = self.df['일자'].iloc[0]
        monthly_dates = pd.concat([pd.Series([first_date]), monthly_last_dates['일자']], ignore_index=True)

        # '일자' 컬럼에서 monthly_dates에 해당하는 값만 필터링
        filtered_df = self.df[self.df['일자'].isin(monthly_dates)]

        return filtered_df
    
    def calculate_excess_return(self):
        # 월간 수익률 계산
        monthly_returns = self.monthly_return()

        # columns_singleindex의 첫 번째 및 두 번째 항목을 사용하여 초과수익률 계산
        fund_column = self.columns_singleindex[0]  # '수정기준가'
        kospi_column = self.columns_singleindex[1]  # 'KOSPI지수'
        
        # '초과수익률' 계산
        monthly_returns['초과수익률'] = monthly_returns[fund_column] - monthly_returns[kospi_column]

        return monthly_returns
    
    def calculate_ytd_values(self, get_monthly_dates):
        ytd_values = {}
        for year in get_monthly_dates['일자'].unique():
            # 해당 연도의 데이터 추출
            year_data = get_monthly_dates[get_monthly_dates['일자'].dt.year == year]

            # 해당 연도 데이터에 대해 누적 수익률 계산
            cumulative_year_data = self.calculate_cumulative_return_for_df(year_data)

            # 계산된 누적 수익률의 마지막 값을 YTD 값으로 사용
            ytd_values[year] = {}
            for column_name in ['수정기준가', 'KOSPI지수', '초과수익률']:
                ytd_values[year][column_name] = cumulative_year_data[column_name].iloc[-1]

        return ytd_values
    
    def create_monthly_calendar_df(self):
        # 월별 수익률 데이터를 가져옴 (초과수익률 포함)
        monthly_returns = self.calculate_excess_return()

        # '일자' 칼럼을 연도와 월로 분리
        monthly_returns[['연도', '월']] = monthly_returns['일자'].str.split('-', expand=True)

        # MultiIndex 생성을 위한 리스트 준비
        index_tuples = []
        for year in monthly_returns['연도'].unique():
            for indicator in ['수정기준가', 'KOSPI지수', '초과수익률']:
                index_tuples.append((year, indicator))

        # MultiIndex 생성
        multi_index = pd.MultiIndex.from_tuples(index_tuples, names=['연도', '지표'])

        # 월별 데이터를 행렬 형태로 재구성
        calendar_df = pd.DataFrame(index=multi_index)
        months = [f"{i}월" for i in range(1, 13)] + ['YTD']  # 1월부터 12월까지와 YTD 포함

        for month in months:
            calendar_df[month] = None  # 초기화

        for _, row in monthly_returns.iterrows():
            year = row['연도']
            month = f"{int(row['월'])}월"
            calendar_df.at[(year, '수정기준가'), month] = row['수정기준가']
            calendar_df.at[(year, 'KOSPI지수'), month] = row['KOSPI지수']
            calendar_df.at[(year, '초과수익률'), month] = row['초과수익률']

        # YTD 값 추가
        ytd_values = self.calculate_ytd_values(self.get_monthly_dates())

        for year in ytd_values.keys():
            for indicator in ['수정기준가', 'KOSPI지수', '초과수익률']:
                calendar_df.at[(year, indicator), 'YTD'] = ytd_values[year][indicator]

        return calendar_df
  

    def monthly_return(self):
        self.get_df_ref(['일자']+self.columns_singleindex)
        self.filter_by_date_range()
        self.convert_to_float(self.columns_singleindex)
        self.fill_zero_with_previous(self.columns_singleindex)
        monthly_filtered_df = self.get_monthly_dates()

        # 월간 수익률 계산
        monthly_returns = monthly_filtered_df[self.columns_singleindex].pct_change().dropna()

        # '일자' 칼럼의 날짜 형식을 '연-월' 형식으로 변경
        monthly_returns['일자'] = monthly_filtered_df['일자'].dt.strftime('%Y-%m')
        
        

        return monthly_returns

보고서와 데이터와의 기간 차이로 인해 오차 발생

보고서의 기간은 2021.07.29 ~ 2023.10.31

데이터의 기간은 2021.07.29 ~ 2023.10.30

In [14]:
m = M8186(fund_code = '100004')
m.period_cumulative_return()


/var/folders/xt/31fyqgg93lb1b81p26rmp09c0000gn/T/ipykernel_44311/4094994272.py:70: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  self.df.fillna(method='ffill', inplace=True)


,펀드,KOSPI,KOSPI200,KOSDAQ,S&P 500
기간,,,,,
1개월,-5.320985,-6.268382,-5.436014,-9.975982,-2.827159
3개월,-5.550120,-11.416161,-10.196785,-17.140543,-9.065673
6개월,3.902052,-7.634528,-5.363597,-10.169311,-0.063797
1년,36.941799,1.858138,4.707517,10.105725,6.812507
2년,30.043045,-22.221512,-20.470049,-23.702800,-9.522776
YTD,22.904183,3.315596,6.131913,11.457551,8.525068
설정이후,28.396716,-28.745008,-28.065846,-27.487956,-5.709922


In [15]:
m = M8186(fund_code = '100004')
m.investment_performance()


,누적수익률,연환산 수익률,변동성,샤프비율,Winning Ratio,MDD
펀드,28.396716,12.578642,19.191235,0.655437,60.072816,18.211615
KOSPI,-28.745008,-12.732922,16.153626,-0.788239,33.252427,34.291454


In [16]:
m = M8186(fund_code = '100004')
m.investment_performance()

,누적수익률,연환산 수익률,변동성,샤프비율,Winning Ratio,MDD
펀드,28.396716,12.578642,19.191235,0.655437,60.072816,18.211615
KOSPI,-28.745008,-12.732922,16.153626,-0.788239,33.252427,34.291454


In [61]:
m = M8186(fund_code = '100004')
m.monthly_return()
m.calculate_excess_return()


,수정기준가,KOSPI지수,일자,초과수익률
2,0.000010,-0.012437,2021-07,0.012447
33,0.018040,-0.000952,2021-08,0.018992
63,-0.026619,-0.040775,2021-09,0.014156
94,-0.003643,-0.031980,2021-10,0.028337
124,-0.034436,-0.044323,2021-11,0.009888
155,0.085971,0.048834,2021-12,0.037137
186,-0.046546,-0.105556,2022-01,0.059010
214,0.004944,0.013457,2022-02,-0.008513
245,0.025232,0.021662,2022-03,0.003570
275,0.053135,-0.022700,2022-04,0.075836


In [56]:
m.get_monthly_dates()

,일자,수정기준가,KOSPI지수
0,2021-07-29,1000.01,3242.65
2,2021-07-31,1000.02,3202.32
33,2021-08-31,1018.06,3199.27
63,2021-09-30,990.96,3068.82
94,2021-10-31,987.35,2970.68
124,2021-11-30,953.35,2839.01
155,2021-12-31,1035.31,2977.65
186,2022-01-31,987.12,2663.34
214,2022-02-28,992.00,2699.18
245,2022-03-31,1017.03,2757.65


In [62]:
m.create_monthly_calendar_df()

IndexError: single positional indexer is out-of-bounds

In [18]:
m.get_monthly_last_dates()

AttributeError: 'M8186' object has no attribute 'get_monthly_last_dates'

In [ ]:
m.get_monthly_returns()

{'2021-07': {'수정기준가_월간수익률': 0.0009999900000990894,
  'KOSPI지수_월간수익률': -1.2437358333461805},
 '2021-08': {'수정기준가_월간수익률': 1.8029459116226487,
  'KOSPI지수_월간수익률': -0.0952434484998433},
 '2021-09': {'수정기준가_월간수익률': -3.024846604754026,
  'KOSPI지수_월간수익률': -4.30929648084514},
 '2021-10': {'수정기준가_월간수익률': 1.0769530010339547,
  'KOSPI지수_월간수익률': -1.6063964387681424},
 '2021-11': {'수정기준가_월간수익률': -3.046851959199024,
  'KOSPI지수_월간수익률': -4.697308438572104},
 '2021-12': {'수정기준가_월간수익률': 6.8486506011662005,
  'KOSPI지수_월간수익률': 2.687500862152218},
 '2022-01': {'수정기준가_월간수익률': -4.655565429046086,
  'KOSPI지수_월간수익률': -10.555639514382145},
 '2022-02': {'수정기준가_월간수익률': 0.49436745279196,
  'KOSPI지수_월간수익률': 1.345678734220929},
 '2022-03': {'수정기준가_월간수익률': 2.522151994435538,
  'KOSPI지수_월간수익률': 2.1662134426010957},
 '2022-04': {'수정기준가_월간수익률': 4.882443375995133,
  'KOSPI지수_월간수익률': -1.6351260105480128},
 '2022-05': {'수정기준가_월간수익률': -1.7374985995443757,
  'KOSPI지수_월간수익률': -0.33951132632047976},
 '2022-06': {'수정기준가_월간수익률': 

In [ ]:
m.get_cumulative_return()

{'수정기준가': 28.396716032839674, 'KOSPI지수': -28.745007941035876}

In [ ]:
m.get_annualized_return()

{'수정기준가': 12.578642417459323, 'KOSPI지수': -12.73292220689089}

In [ ]:
m.get_volatility()

{'수정기준가': 19.19123518240374, 'KOSPI지수': 16.153626441399805}

In [ ]:
m.get_sharpe_ratio()

{'수정기준가': 0.6554368334244874, 'KOSPI지수': -0.7882392386057623}

In [ ]:
m.get_winning_ratio()

{'수정기준가': 0.656498673740053, 'KOSPI지수': 0.363395225464191}

In [ ]:
m.get_mdd()

{'수정기준가': 0.18211614809343543, 'KOSPI지수': 0.3429145403886136}

In [ ]:
positive_counts = (m.daily_returns['KOSPI지수'] < 0).sum()

print(positive_counts)

279


In [ ]:
zero_rows = (m.daily_returns['수정기준가'] == 0) & (m.daily_returns['KOSPI지수'] == 0)

# 해당 조건을 만족하는 행의 수 계산
count_zero_rows = zero_rows.sum()

print(count_zero_rows)

70


In [ ]:
t = M8186(fund_code = '100004')
t.get_df_ref(['일자', '수정기준가', 'KOSPI지수', 'KOSPI200지수', 'KOSDAQ지수'])
t.filter_by_date_range()
t.convert_to_float(['수정기준가', 'KOSPI지수', 'KOSPI200지수', 'KOSDAQ지수'])
t.fill_zero_with_previous(['수정기준가', 'KOSPI지수', 'KOSPI200지수', 'KOSDAQ지수'])

df = t.df

In [ ]:
df

,일자,수정기준가,KOSPI지수,KOSPI200지수,KOSDAQ지수
0,2021-07-29,1000.01,3242.65,429.49,1044.13
1,2021-07-30,1000.02,3202.32,423.91,1031.14
2,2021-07-31,1000.02,3202.32,423.91,1031.14
3,2021-08-01,1000.03,3202.32,423.91,1031.14
4,2021-08-02,1000.01,3223.04,426.79,1037.80
...,...,...,...,...,...
819,2023-10-26,1271.60,2299.08,307.75,743.85
820,2023-10-27,1280.08,2302.81,308.53,748.49
821,2023-10-28,1280.12,2302.81,308.53,748.49
822,2023-10-29,1280.16,2302.81,308.53,748.49


In [ ]:
columns_except_date = df.columns.difference(['일자','수정기준가'])

# 선택된 칼럼에 대해 퍼센트 변화율 계산
diff_df = df[columns_except_date].diff()
# 첫 행의 NaN 값을 0으로 대체
diff_df.iloc[0] = 0

print(diff_df)

     KOSDAQ지수  KOSPI200지수  KOSPI지수
0        0.00        0.00     0.00
1      -12.99       -5.58   -40.33
2        0.00        0.00     0.00
3        0.00        0.00     0.00
4        6.66        2.88    20.72
..        ...         ...      ...
819    -26.99       -8.74   -64.09
820      4.64        0.78     3.73
821      0.00        0.00     0.00
822      0.00        0.00     0.00
823      8.63        0.42     7.74

[824 rows x 3 columns]


In [ ]:
zero_rows = (diff_df == 0).all(axis=1)

# 해당 조건을 만족하는 행의 수 계산
count_zero_rows = zero_rows.sum()

print(count_zero_rows)

271


In [ ]:
positive_counts = (diff_df > 0).sum()

print(positive_counts)

KOSDAQ지수      290
KOSPI200지수    263
KOSPI지수       274
수정기준가         495
dtype: int64


전일대비수익률 칼럼은 사용할 수 없을것이란 결론

In [ ]:
k = M8186(fund_code = '100004')
dfk = k.get_df_ref(['일자', '전일대비수익률', '전일대비BM수익률(%)'])
columns_except_date = dfk.columns.difference(['일자'])

# 선택된 칼럼에 대해 퍼센트 변화율 계산
dfk = dfk[columns_except_date]
dfk

,전일대비BM수익률(%),전일대비수익률
0,0.18,0.00
1,-1.24,0.00
2,0.00,0.00
3,0.00,0.00
4,0.65,0.00
...,...,...
819,-2.71,-1.25
820,0.16,0.67
821,0.00,0.00
822,0.00,0.00


In [ ]:
zero_rows = (dfk == 0).all(axis=1)

# 해당 조건을 만족하는 행의 수 계산
count_zero_rows = zero_rows.sum()

print(count_zero_rows)

270


In [ ]:
positive_counts = (dfk > 0).sum()

print(positive_counts)

전일대비BM수익률(%)    275
전일대비수익률         292
dtype: int64
